In [2]:
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from linkedin_api import Linkedin
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import sqlite3



In [14]:
connection = sqlite3.connect('labelled_data.db')

# Write a SQL query to select the data
query = "SELECT * FROM labelled_job_postings"

# Load the data into a DataFrame
df = pd.read_sql_query(query, connection)

# Close the database connection
connection.close()

# Display the DataFrame


In [15]:
print(len(df))

row_counts = df.groupby(['company', 'description', 'title']).size().reset_index(name='count').sort_values('count', ascending=False)
print(row_counts['count'])


9458
988     141
893     135
987      77
922      69
990      67
       ... 
885       1
756       1
125       1
1333      1
1021      1
Name: count, Length: 1386, dtype: int64


In [16]:
df = df.drop_duplicates(subset=['company', 'description', 'title'], keep='first')
print(len(df))
print(df.head())


1387
   id job_posting_id                     title  \
0   1     4038627932             PHP Developer   
1   2     4041491685             PHP Developer   
2   3     4039079429  Python Software Engineer   
3   4     4039504748       Full Stack Engineer   
4   5     4040799491     C++ Software Engineer   

                                             company  \
0                                          Odd Shoes   
1  Better Placed Ltd - A Sunday Times Top 10 Empl...   
2                                                CFC   
3                                        Two Circles   
4             KO2 Embedded Recruitment Solutions Ltd   

                           location  \
0                    United Kingdom   
1                    United Kingdom   
2       London Area, United Kingdom   
3   London, England, United Kingdom   
4  Reading, England, United Kingdom   

                                         description  job_site  \
0  We all remember that favourite teacher from sc...  Li

In [17]:
# Connect to the SQLite database
conn = sqlite3.connect('labelled_data.db')
cursor = conn.cursor()

# Delete duplicate rows while keeping the first occurrence
cursor.execute('''
    DELETE FROM labelled_job_postings
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM labelled_job_postings
        GROUP BY company, description, title
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Duplicate records removed successfully!")

Duplicate records removed successfully!


In [18]:
# Connect to the SQLite database
conn = sqlite3.connect('labelled_data.db')
cursor = conn.cursor()

# Delete duplicate rows while keeping the first occurrence
cursor.execute('''
    DELETE FROM labelled_job_postings
    WHERE sponsorship_available is null
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Duplicate records removed successfully!")

Duplicate records removed successfully!


In [37]:
df = pd.read_csv("ai_generated_data/uncertain/data.csv")

data = [
    (row['Job Title'], row['Company Name'], row['Location'], row['Job Description'], 'Uncertain')
    for index, row in df.iterrows()
]

conn = sqlite3.connect('labelled_data.db')
cursor = conn.cursor()
cursor.executemany('''
                        INSERT INTO labelled_job_postings(title, company, location, description, sponsorship_available)
                        VALUES (?, ?, ?, ?, ?)
                    ''', data
    )
conn.commit()
conn.close()

In [38]:
# Connect to the SQLite database
conn = sqlite3.connect('labelled_data.db')
cursor = conn.cursor()

# Delete duplicate rows while keeping the first occurrence
cursor.execute('''

    DELETE FROM labelled_job_postings
    WHERE id IN (
        SELECT id FROM (
            SELECT id
            FROM labelled_job_postings
            WHERE sponsorship_available = 'Sponsorship Provided'
            ORDER BY id DESC
            LIMIT 12
        ) subquery
    )         
    
  
''')

# Commit the changes and close the connection
conn.commit()
conn.close()